In [ ]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from math import radians
from sklearn.metrics.pairwise import haversine_distances
import numpy as np

df = pd.read_csv('step.csv',encoding='latin1')

# distance function
def calculate_distance(coord1, coord2):
    coord1 = [radians(float(x)) for x in coord1.split(',')]
    coord2 = [radians(float(x)) for x in coord2.split(',')]
    # Calculate distance using haversine formula
    distance = haversine_distances([coord1, coord2])
    # Convert distance from radians to kilometers
    distance_km = distance * 6371000 / 1000  # Earth radius in kilometers
    return distance_km[0][1]


def train_knn_model(dataset):
    X = dataset[['Rating', 'Coordinates']].copy()
    X['Latitude'] = X['Coordinates'].apply(lambda x: float(x.split(',')[0]))
    X['Longitude'] = X['Coordinates'].apply(lambda x: float(x.split(',')[1]))
    X = X[['Rating', 'Latitude', 'Longitude']]
    knn_model = NearestNeighbors(n_neighbors=5, metric='euclidean')
    knn_model.fit(X)
    return knn_model

# Recommendation Function
def recommend_places_ml(place_name, knn_model, dataset):
    place_data = dataset[dataset['DestinationName'] == place_name].iloc[0]
    query_point = [[place_data['Rating']] + [float(coord) for coord in place_data['Coordinates'].split(',')]]
    distances, indices = knn_model.kneighbors(query_point)
    recommendations = dataset.iloc[indices[0][1:]]
    # Calculate distances from query point to recommended places
    query_coords = [float(coord) for coord in place_data['Coordinates'].split(',')]
    recommendations['Distance'] = recommendations['Coordinates'].apply(lambda x: calculate_distance(x, ','.join(map(str, query_coords))))
    return recommendations[['DestinationName', 'Rating', 'Distance']]

# Usage
knn_model = train_knn_model(df)
place_name = "Qutub Minar"
recommended_places = recommend_places_ml(place_name, knn_model, df)
print("Recommended places around", place_name, ":")
print(recommended_places)


Recommended places around Qutub Minar :
           DestinationName  Rating  Distance
39           Gandhi Smriti    0.85  1.551563
37            Lodhi Garden    0.85  4.289356
36  Gurudwara Bangla Sahib    0.88  4.522220
40              Dilli Haat    0.80  3.312473


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
<ipython-input-5-a2c916cb434e>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommendations['Distance'] = recommendations['Coordinates'].apply(lambda x: calculate_distance(x, ','.join(map(str, query_coords))))


In [ ]:
!pip install haversine